<h1>IBM Applied Data Science Capstone Course by Coursera</h1>  
<h3>Week 3 Part 1 & 2 & 3</h3>   
<h4>Task 1 : Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.</h4>
<h4>Task 2 : To get the latitude and the longitude coordinates of each neighborhood.   
<h4>Task 3 : Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.  


<h3>Step 1: Importinng all required libraries for the 3 parts of code 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import urllib
import bs4
print("Libraries imported.")

Libraries imported.


"<h3> Using Beautiful soup library to parse data from the wikipedia page

In [2]:
#url link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Parsing data from the url link using Beautiful soup package
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs4.BeautifulSoup(source,'html.parser')

#Identifying table and its rows and saving them 
table = soup.find('table')
table_rows = table.find_all('tr')

#Create empty list and save the data from the page into it
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

<h3> Creating Dataframe to store the data from the list into columns and naming them

In [3]:
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


<h3> Removing rows with value of " Not assigned"

In [4]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


<h3> More than one neighbour can exist in same postal code

In [5]:
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<h3> " If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough." - from question

In [6]:
df.loc[df['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<h3>Shape of the dataframe as asked

In [7]:
df.shape

(103, 3)

<h3> Part 2 : Getting Latitude and Longitude 

In [8]:
!conda install -c conda-forge geocoder --yes 
import geocoder


Solving environment: done

# All requested packages already installed.



<h3> Reading the geospatial coordinates csv file into coordinates data frame

In [9]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3> Renaming the column name "Postal Code" to "PostalCode" so merge is proper

In [10]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>Merging the data  frames to produce required lattitude and longitude values for neighbourhood

In [19]:
toronto_df_new = df.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h3> Checking the dataframe from question. Please ignore another column of Neighborhood

In [12]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Neighbourhood
0,M5G,Downtown Toronto,NaN,43.657952,-79.387383,Central Bay Street
1,M2H,North York,NaN,43.803762,-79.363452,Hillcrest Village
2,M4B,East York,NaN,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,NaN,43.744734,-79.239476,Scarborough Village
4,M4G,East York,NaN,43.709060,-79.363452,Leaside
5,M4M,East Toronto,NaN,43.659526,-79.340923,Studio District
6,M1R,Scarborough,NaN,43.750072,-79.295849,"Wexford, Maryvale"
7,M9V,Etobicoke,NaN,43.739416,-79.588437,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,NaN,43.756303,-79.565963,Humber Summit
9,M5V,Downtown Toronto,NaN,43.628947,-79.394420,"CN Tower, King and Spadina, Railway Lands, Har..."


<h3> Checking Number of borough and Neighborhood

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df['Borough'].unique()), df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


<h2>PART 3 :: Segmenting and Clustering the neighbourhoods in toronto

In [14]:
# Use geopy library to get the latitude and longitude values of Toronto Ontario
address = 'Toronto Ontario, TO'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto Ontario are 43.6534817, -79.3839347.


In [20]:
# create map of Toronto Ontario using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [21]:

# just focus on Scarborough
scarborough_data = toronto_df_new[toronto_df_new['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [22]:
address = 'Scarborough, Toronto Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [23]:

# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [25]:
# Foursquare Credentials
CLIENT_ID = 'R0UKCI54LFVBUI0P3KAITBZR3GCCARXW5LUHFVYSFYVIQU3M' # your Foursquare ID
CLIENT_SECRET = 'SJTSO1DROW3RYJ0SK2QJ23DM3A3IICAOHQD5RHFFSXJQVC43' # your Foursquare Secret
VERSION = '20200523' # Foursquare API version
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: R0UKCI54LFVBUI0P3KAITBZR3GCCARXW5LUHFVYSFYVIQU3M
CLIENT_SECRET:SJTSO1DROW3RYJ0SK2QJ23DM3A3IICAOHQD5RHFFSXJQVC43


In [26]:

# explore the first neighborhood in our dataframe.
scarborough_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [27]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [28]:

# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=R0UKCI54LFVBUI0P3KAITBZR3GCCARXW5LUHFVYSFYVIQU3M&client_secret=SJTSO1DROW3RYJ0SK2QJ23DM3A3IICAOHQD5RHFFSXJQVC43&v=20200523&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec8e7a178a484001b6cdb93'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:

# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


2 venues were returned by Foursquare.


<h3>Exploring Neighbourhoods in scarborough

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
# get scarborough venues
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

print(scarborough_venues.shape)
scarborough_venues.head(10)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
(95, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Wood Floor Polishing Inc,43.766500,-79.185207,Moving Target
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [36]:

# find out how many unique categories
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",8,8,8,8,8,8


In [37]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


<h3> Analyze Each Neighborhood

In [38]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
scarborough_onehot.shape

(95, 56)

In [40]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.25,0.25,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.250000,0.00,0.0,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.2,0.000,0.000000,0.000000,0.2,0.0,0.000000,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.200000,0.00,0.0,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.125,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.0,0.000000,0.125,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.071429,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.071429,0.071429,0.0,0.0,0.071429,0.000,0.000000,0.000000,0.0,0.071429,0.071429,0.071429,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.071429,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.071429,0.0,0.0,0.071429,0.142857,0.0,0.0,0.000,0.000000,0.071429,0.000000,0.00,0.0,0.071429,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.333333,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.333333,0.00,0.0,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.200000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.40,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.100,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.1,0.00,0.100,0.000000,0.00,0.00,0.00,0.000,0.1,0.000,0.00,0.000000,0.000,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.125000,0.0,0.125000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.125,0.000000,0.00,0.00,0.00,0.125,0.0,0.125,0.00,0.000000,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0
8,"Kennedy Park, Ionvie

In [41]:

scarborough_grouped.shape


(16, 56)

In [42]:

# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink   0.2
1  General Entertainment   0.2
2                   Café   0.2
3                   Farm   0.2
4        College Stadium   0.2


----Cedarbrae----
                 venue  freq
0               Bakery  0.12
1                 Bank  0.12
2      Thai Restaurant  0.12
3  Fried Chicken Joint  0.12
4   Athletics & Sports  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                 venue  freq
0          Pizza Place  0.14
1    Convenience Store  0.07
2   Chinese Restaurant  0.07
3          Gas Station  0.07
4  Fried Chicken Joint  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1         Skat

In [43]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
7,"Guildwood, Morningside, West Hill",Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
8,"Kennedy Park, Ionview, East Birchmount Park",Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
9,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


<h4>4.Cluster Neighbourhoods

In [55]:

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 3, 3, 3, 1, 3, 3, 3, 3, 0, 4, 2, 4, 3, 3, 3], dtype=int32)

In [62]:

neighborhoods_venues_sorted.head(20)


,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,3,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,1,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
5,3,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,3,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
7,3,"Guildwood, Morningside, West Hill",Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
8,3,"Kennedy Park, Ionview, East Birchmount Park",Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
9,0,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


In [63]:
scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store


In [70]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

<h3> Examining the CLUSTERS

In [65]:

# Cluster 1 
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


In [66]:

# Cluster 2
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,1.0,American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
12,Scarborough,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [67]:

# Cluster 3
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


In [68]:

# Cluster 4
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3.0,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,Scarborough,3.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,Scarborough,3.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
6,Scarborough,3.0,Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
7,Scarborough,3.0,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
9,Scarborough,3.0,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,3.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
11,Scarborough,3.0,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,3.0,Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
15,Scarborough,3.0,Fast Food Restaurant,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Discount Store,Electronics Store,Pharmacy,Pizza Place,Grocery Store,Breakfast Spot


In [69]:
# Cluster 5
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4.0,Playground,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
14,Scarborough,4.0,Playground,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


<h2> Hence Done.Most of the neighbourhoods fall into cluster four where most of the common venues are Restaurants or food retails.